In [10]:
import numpy as np
import os
import glob
from os.path import isfile
import keras
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Bidirectional, LSTM, Dropout, Activation, GRU
from keras.layers import Conv2D, concatenate, MaxPooling2D, Flatten, Embedding, Lambda

from keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from keras import backend as K
from keras.utils import np_utils
from keras.optimizers import Adam, RMSprop
from keras import regularizers

import librosa
import librosa.display
import matplotlib.pyplot as plt
%matplotlib inline


In [11]:
dict_class ={'Not Parkinsons':0, 'Parkinsons':1}

reverse_map = {v: k for k, v in dict_class.items()}
print(reverse_map)

{0: 'Not Parkinsons', 1: 'Parkinsons'}


In [13]:
#import spectrograms
spect = []
spect_id = []
park_id = []
#for j in ['Parkinsons','NonParkinsons']:

for q in ['alda','Bannister','Buck','Bush', 'Connolly','Isakson','JesseJackson','Kinsley',
             'MJF','Serrano','Shapiro']:
    file = []
    for root, dirs, files in os.walk('Specs/Parkinsons/'+q):
        file += glob.glob(os.path.join(root,'*.png'))
    for i in file:
        x = plt.imread(i) 
        spect.append(x)
        spect_id.append(q)
        park_id.append(1)       
for q in ['Bray','Crystal','Cummberbatch','Dispenza', 'Letterman','Phillips','Prevlen','Spurlock',
             'Treasure','Tyson','Urban','Walker']:
    file = [] 
    for root, dirs, files in os.walk('Specs/NonParkinsons/'+q):
        file += glob.glob(os.path.join(root,'*.png'))
    for i in file:
        x = plt.imread(i) 
        spect.append(x)
        spect_id.append(q)
        park_id.append(0)
    
x_train = spect
y_train = spect_id
YTrain = park_id


In [16]:
#create subset of data for test and train
import random
print(len(x_train))

test_indices = random.sample(range(len(x_train)), 630)
train_indices = []
for i in range(len(x_train)):
    if i not in test_indices:
        train_indices.append(i)

x_new_train = []
y_new_train = []
YNEWTRAIN = []
x_test = []
y_test = []
YTEST = []

for i in test_indices:
    x_test.append(x_train[i])
    y_test.append(y_train[i])
    YTEST.append(YTrain[i])

for i in train_indices:
    x_new_train.append(x_train[i])
    y_new_train.append(y_train[i])
    YNEWTRAIN.append(YTrain[i])

2105


In [64]:
X_train = np.asarray(x_new_train)
Y_train = np.asarray(YNEWTRAIN)
X_valid = np.asarray(x_test)
Y_valid = np.asarray(YTEST)

In [22]:
batch_size = 32
num_classes = 2
n_features = X_train.shape[2] 
n_time = X_train.shape[1]


In [60]:
nb_filters1=16 
nb_filters2=32 
nb_filters3=64
nb_filters4=64
nb_filters5=64
ksize = (4,4)
pool_size_1= (2,2) 
pool_size_2= (2,2)
pool_size_3 = (2,2)

dropout_prob = 0.20
dense_size1 = 288
lstm_count = 64
num_units = 120

BATCH_SIZE = 64
EPOCH_COUNT = 50
L2_regularization = 0.001

def conv_recurrent_model_build(model_input):
    print('Building model...')
    layer = model_input
    
    ### Convolutional blocks
    conv_1 = Conv2D(filters = nb_filters1, kernel_size = ksize, strides=1,
                      padding= 'valid', activation='relu', name='conv_1', input_shape=(288,432,4))(layer)
    pool_1 = MaxPooling2D(pool_size_1)(conv_1)

    conv_2 = Conv2D(filters = nb_filters2, kernel_size = ksize, strides=1,
                      padding= 'valid', activation='relu', name='conv_2')(pool_1)
    pool_2 = MaxPooling2D(pool_size_1)(conv_2)

    conv_3 = Conv2D(filters = nb_filters3, kernel_size = ksize, strides=1,
                      padding= 'valid', activation='relu', name='conv_3')(pool_2)
    pool_3 = MaxPooling2D(pool_size_1)(conv_3)
    
    
    conv_4 = Conv2D(filters = nb_filters4, kernel_size = ksize, strides=1,
                      padding= 'valid', activation='relu', name='conv_4')(pool_3)
    pool_4 = MaxPooling2D(pool_size_2)(conv_4)
    
    
    conv_5 = Conv2D(filters = nb_filters5, kernel_size = ksize, strides=1,
                      padding= 'valid', activation='relu', name='conv_5',input_shape=(288,432,4))(pool_4)
    pool_5 = MaxPooling2D(pool_size_2)(conv_5)

    flatten1 = Flatten()(pool_5)
    ### Recurrent Block
    
    # Pooling layer
    pool_lstm1 = MaxPooling2D(pool_size_3, name = 'pool_lstm')(layer)
    
    # Embedding layer

    squeezed = Lambda(lambda x: K.squeeze(x, axis= -1))(pool_lstm1)
#     flatten2 = K.squeeze(pool_lstm1, axis = -1)
#     dense1 = Dense(dense_size1)(flatten)
    
    # Bidirectional GRU
    lstm = Bidirectional(GRU(lstm_count))(squeezed)  #default merge mode is concat
    
    # Concat Output
    concat = concatenate([flatten1, lstm], axis=-1, name ='concat')
    
    ## Softmax Output
    output = Dense(num_classes, activation = 'softmax', name='preds')(concat)
    
    model_output = output
    model = Model(model_input, model_output)
    
#     opt = Adam(lr=0.001)
    opt = RMSprop(lr=0.0005)  # Optimizer
    model.compile(
            loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy']
        )
    
    print(model.summary())
    return model

In [70]:
def train_model(x_train, y_train, x_val, y_val):
    
    n_frequency = 432
    n_frames = 288
    #reshape and expand dims for conv2d
#     x_train = x_train.reshape(-1, n_frequency, n_frames)
    x_train = np.expand_dims(x_train, axis = -1)
    
#     x_val = x_val.reshape(-1, n_frequency, n_frames)
    x_val = np.expand_dims(x_val, axis = -1)
    
    
    #input_shape = (n_frames, n_frequency,4)
    input_shape = (288, 432,4)
    model_input = Input(input_shape, name='input')
    
    model = conv_recurrent_model_build(model_input)
    
#     tb_callback = TensorBoard(log_dir='./logs/4', histogram_freq=1, batch_size=32, write_graph=True, write_grads=False,
#                               write_images=False, embeddings_freq=0, embeddings_layer_names=None,
#                               embeddings_metadata=None)
    checkpoint_callback = ModelCheckpoint('./models/parallel/weights.best.h5', monitor='val_acc', verbose=1,
                                          save_best_only=True, mode='max')
    
    reducelr_callback = ReduceLROnPlateau(
                monitor='val_acc', factor=0.5, patience=10, min_delta=0.01,
                verbose=1
            )
    callbacks_list = [checkpoint_callback, reducelr_callback]

    # Fit the model and get training history.
    print('Training...')
    history = model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCH_COUNT,
                        validation_data=(x_val, y_val), verbose=1, callbacks=callbacks_list)

    return model, history

In [25]:
def show_summary_stats(history):
    # List all data in history
    print(history.history.keys())

    # Summarize history for accuracy
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

    # Summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [71]:
model, history  = train_model(X_train, Y_train, X_valid, Y_valid)

Building model...


ValueError: Can not squeeze dim[3], expected a dimension of 1, got 4 for 'lambda_15/Squeeze' (op: 'Squeeze') with input shapes: [?,144,216,4].

In [43]:
print(X_train.shape)

(1475, 288, 432, 4)


In [ ]:
show_summary_stats(history)

In [ ]:
from sklearn.metrics import classification_report

y_true = np.argmax(y_valid, axis = 1)
X_valid = np.expand_dims(X_valid, axis = -1)
y_pred = model.predict(X_valid)
y_pred = np.argmax(y_pred, axis=1)
labels = [0,1,2,3,4,5,6,7]
target_names = dict_genres.keys()

print(y_true.shape, y_pred.shape)
print(classification_report(y_true, y_pred, target_names=target_names))

In [ ]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_true, y_pred))

In [50]:
npzfile = np.load('shuffled_train.npz')
print(npzfile.files)
X_train = npzfile['arr_0']
y_train = npzfile['arr_1']
print(X_train.shape, y_train.shape)

['arr_0', 'arr_1']
(6394, 640, 128) (6394, 8)


In [ ]:
num = 5300
spectogram = X_train[num]
genre = np.argmax(y_train[num])
print(reverse_map[genre])
plt.figure(figsize=(10, 5))
librosa.display.specshow(spectogram.T, y_axis='mel', x_axis='time')
plt.colorbar(format='%+2.0f dB')
plt.title('Test Melspectogram')
plt.tight_layout()
